<a href="https://colab.research.google.com/github/rulomak/Proyecto_4-clasificacion-de-texto-scikit-learn/blob/main/Proyecto_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Vincular disco
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

df_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
df_review.value_counts('sentiment')

sentiment
negative    25000
positive    25000
dtype: int64

## Reduccion de datos 

In [7]:
df_review_small = df_review[:5000]

In [8]:
df_review_small.value_counts('sentiment')

sentiment
negative    2532
positive    2468
dtype: int64

## Separando data para entrenar (train) y testear (test)


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_review_small, test_size=0.30, random_state=42)

In [10]:
# defino las variables  para cada uno  
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

## Transformar data de texto a data numerica

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)   
test_x_vector = tfidf.transform(test_x)  

In [12]:
train_x_vector

<3500x32758 sparse matrix of type '<class 'numpy.float64'>'
	with 310195 stored elements in Compressed Sparse Row format>

## Support Vector Machines (SVM)

In [13]:
from sklearn.svm import SVC


In [14]:

svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

# Evaluacion del Modelo

In [15]:
print(svc.score(test_x_vector, test_y))

0.8593333333333333


## F1 Score


In [16]:
from sklearn.metrics import f1_score                  

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

array([0.85616905, 0.86236138])

## Reporte de Clasificacion

In [17]:
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                      labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.84      0.88      0.86       717
    negative       0.88      0.84      0.86       783

    accuracy                           0.86      1500
   macro avg       0.86      0.86      0.86      1500
weighted avg       0.86      0.86      0.86      1500



## Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, svc.predict(test_x_vector),
                 labels=['positive', 'negative'])

array([[628,  89],
       [122, 661]])

## Testeo

In [25]:
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['"I did not like this movie at all I gave this movie away"'])))
print(svc.predict(tfidf.transform(["movie disgust"])))

['positive']
['positive']
['negative']
['negative']
